In [52]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors as mcolors
import os
from shapely.geometry import Polygon
import rasterio
import glob
import pandas as pd
from rasterio.warp import calculate_default_transform, reproject, Resampling

#检查当前工作目录
print(os.getcwd())

#修改当前工作目录
os.chdir("/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse1.CCFR calculation")

#检查当前工作目录
print(os.getcwd())

/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse1.CCFR calculation
/public/home/jiaqizhang/JiaqiZhang/project-CCFR/02.Analysis and figures/01.notebook/Analyse1.CCFR calculation


In [53]:
# 读取shapefile数据
shp_file_path = "/public/home/jiaqizhang/JiaqiZhang/project-CCFR/01.Processed data/11.GDP/gridnet_GDP.shp"
gridnet = gpd.read_file(shp_file_path, engine="pyogrio")

# 删除不需要的列，避免重复项
columns_to_delete = ["TF_risk", "CF_risk", "RF_risk",'count']
gridnet = gridnet.drop(columns=[col for col in columns_to_delete if col in gridnet.columns], errors='ignore')
# 输出信息
print("坐标参考系统 (CRS):", gridnet.crs)
print("列名:", gridnet.columns)
print("数据行数:", len(gridnet))

坐标参考系统 (CRS): EPSG:4326
列名: Index(['A_CFrp0005', 'A_CFrp0010', 'A_CFrp0025', 'A_CFrp0050', 'A_CFrp0100',
       'A_CFrp0250', 'A_CFrp0500', 'A_CFrp1000', 'A_RFrp0005', 'A_RFrp0010',
       'A_RFrp0025', 'A_RFrp0050', 'A_RFrp0100', 'A_RFrp0250', 'A_RFrp0500',
       'A_RFrp1000', 'D_CFrp0005', 'D_CFrp0010', 'D_CFrp0025', 'D_CFrp0050',
       'D_CFrp0100', 'D_CFrp0250', 'D_CFrp0500', 'D_CFrp1000', 'D_RFrp0005',
       'D_RFrp0010', 'D_RFrp0025', 'D_RFrp0050', 'D_RFrp0100', 'D_RFrp0250',
       'D_RFrp0500', 'D_RFrp1000', 'FDD_km-1', 'FH_rp0005', 'FH_rp0010',
       'FH_rp0025', 'FH_rp0050', 'FH_rp0100', 'FH_rp0250', 'FH_rp0500',
       'FH_rp1000', 'MangA_km2', 'T2m', 'T_rivLC_km', 'Tp', 'V_CFrp0005',
       'V_CFrp0010', 'V_CFrp0025', 'V_CFrp0050', 'V_CFrp0100', 'V_CFrp0250',
       'V_CFrp0500', 'V_CFrp1000', 'V_RFrp0005', 'V_RFrp0010', 'V_RFrp0025',
       'V_RFrp0050', 'V_RFrp0100', 'V_RFrp0250', 'V_RFrp0500', 'V_RFrp1000',
       'area_km2', 'area_m2', 'dem', 'net_ID', 'saltA_km2', 

In [54]:
#由于人口数据存在很小的数据（0.0003）因此取整后计算人均GDP
# 将 GDP 和 population 为 0 的值设为 NaN
gridnet['GDP'] = gridnet['GDP'].replace(0, np.nan)
gridnet['population'] = gridnet['population'].fillna(0).astype(int)
gridnet['population'] = gridnet['population'].replace(0, np.nan)

# 计算人均GDP（GDP / population），自动跳过 NaN 值
gridnet['GDP_cap']=gridnet['GDP']/gridnet['population']

print(gridnet[['GDP_cap','GDP','population']].head())



   GDP_cap       GDP  population
0      NaN       NaN         3.0
1      NaN  1.751324         NaN
2      NaN  1.353296         NaN
3      NaN       NaN        23.0
4      NaN       NaN        12.0


In [55]:
# 按倒数GDP列计算10分位数分组
gridnet['GDPcap_qtl'] = pd.qcut(gridnet['GDP_cap'], 10, labels=np.linspace(0.1, 1, 10))
gridnet['GDPcap_qtl'] = gridnet['GDPcap_qtl'].astype(float)

# 将缺失值（NaN）填充为 0（表示最低等级）
gridnet['GDPcap_qtl'] = gridnet['GDPcap_qtl'].fillna(0)
print(gridnet[['GDPcap_qtl']].head())

   GDPcap_qtl
0         0.0
1         0.0
2         0.0
3         0.0
4         0.0


In [56]:
gridnet['GDP_capinv']= 1/gridnet['GDP_cap']
gridnet['G_capinv_q'] = pd.qcut(gridnet['GDP_capinv'], 10, labels=np.linspace(0.1, 1, 10))

# 将缺失值（NaN）填充为 0.1（表示最低等级）
gridnet['G_capinv_q'] = gridnet['G_capinv_q'].fillna(1)

gridnet['G_capinv_q'] = gridnet['G_capinv_q'].astype(float)
print(gridnet[['G_capinv_q']].head())

   G_capinv_q
0         1.0
1         1.0
2         1.0
3         1.0
4         1.0


In [57]:
#计算暴露和脆弱性（GDP*pop），划分为10个quantiles，分别赋值0.1-1，最后将GDP为空值区域的脆弱性设置为0.1。
gridnet['GDP'] = gridnet['GDP'].replace(0, np.nan)
# 按倒数GDP列计算10分位数分组
gridnet['GDP_qtl'] = pd.qcut(gridnet['GDP'], 10, labels=np.linspace(0.1, 1, 10))

gridnet['GDP_qtl'] = gridnet['GDP_qtl'].astype(float)

# 将缺失值（NaN）填充为 0.1（表示最低等级）
gridnet['GDP_qtl'] = gridnet['GDP_qtl'].fillna(0)
print(gridnet[['GDP_qtl']].head())

   GDP_qtl
0      0.0
1      0.3
2      0.2
3      0.0
4      0.0


In [58]:

# 计算GDP的倒数列
gridnet['GDP_inv'] = 1 / gridnet['GDP']
gridnet['GDP_inv'] = gridnet['GDP_inv'].replace(0, np.nan)

# 按倒数GDP列计算10分位数分组
gridnet['GDP_invqtl'] = pd.qcut(gridnet['GDP_inv'], 10, labels=np.linspace(0.1, 1, 10))

# 将缺失值（NaN）填充为 1
gridnet['GDP_invqtl'] = gridnet['GDP_invqtl'].fillna(1)
gridnet['GDP_invqtl'] = gridnet['GDP_invqtl'].astype(float)
print(gridnet[['GDP_invqtl']].head())

   GDP_invqtl
0         1.0
1         0.8
2         0.9
3         1.0
4         1.0


In [59]:
# 定义分位数标签（0.1到1.0）
labels = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]  # [0.1, 0.2, ..., 1.0]
gridnet['TF_Hazard'] = gridnet['TF_Hazard'].replace(0, np.nan)
# 分箱操作（按分位数均分）
gridnet['CCFH_qtl'] = pd.qcut(
    gridnet['TF_Hazard'],
    q=10,                    # 10等分
    labels=labels,           # 分配标签
    # duplicates='drop'        # 避免重复边界报错
)

# 转换为数值类型
gridnet['CCFH_qtl'] = gridnet['CCFH_qtl'].astype(float)
gridnet['CCFH_qtl'] = gridnet['CCFH_qtl'].fillna(0)
print(gridnet[['CCFH_qtl']].head())


   CCFH_qtl
0       0.3
1       0.3
2       0.1
3       0.6
4       0.5


In [60]:
# 定义分位数标签（0.1到1.0）
labels = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]  # [0.1, 0.2, ..., 1.0]

# 分箱操作（按分位数均分）
gridnet['pop_qtl'] = pd.qcut(
    gridnet['population'],
    q=10,                    # 10等分
    labels=labels,           # 分配标签
    # duplicates='drop'        # 避免重复边界报错
)

# 转换为数值类型
gridnet['pop_qtl'] = gridnet['pop_qtl'].astype(float)
gridnet['pop_qtl'] = gridnet['pop_qtl'].fillna(0)
print(gridnet[['pop_qtl']].head())


   pop_qtl
0      0.1
1      0.0
2      0.0
3      0.2
4      0.2


In [ ]:
#CCFR under two scenarios

gridnet['CCFRtake'] = (gridnet['CCFH_qtl']* gridnet['GDP_qtl']) ** (1/2) 
gridnet['CCFRaverse'] = (gridnet['CCFH_qtl']* gridnet['pop_qtl'] * gridnet['G_capinv_q'] ) ** (1/3)  # 开三次方

print("风险值计算完成")
# 查看矢量的列名
print(gridnet.columns)

In [62]:
# 导出为 GeoDataFrame（如果已经是 GeoDataFrame 类型，则无需再转换）
gridnet.to_file('/public/home/jiaqizhang/JiaqiZhang/project-CCFR/01.Processed data/11.GDP/gridnet_GDP_CCFR.shp', driver='ESRI Shapefile')##################################modify here
gridnet.to_excel('/public/home/jiaqizhang/JiaqiZhang/project-CCFR/01.Processed data/11.GDP/gridnet_GDP_CCFR.xlsx', index=False)
